<a href="https://colab.research.google.com/github/krithika-srinivasan/kaggle-ranzcr-clip-catheter-line-classification/blob/main/kaggle_catheter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade --force-reinstall --no-deps kaggle

from google.colab import files
files.upload()

! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/

! chmod 600 ~/.kaggle/kaggle.json

! mkdir data
! cd data

! kaggle competitions download -c ranzcr-clip-catheter-line-classification

! unzip ranzcr-clip-catheter-line-classification.zip

     |████████████████████████████████| 61kB 4.9MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.10-cp37-none-any.whl size=73269 sha256=d41e09a5b8688f34f6d7f5fb3d9baa042380ae45b4aab95c5e539964a6edc7fa
  Stored in directory: /root/.cache/pip/wheels/3a/d1/7e/6ce09b72b770149802c653a02783821629146983ee5a360f10
Successfully built kaggle
  Found existing installation: kaggle 1.5.10
    Uninstalling kaggle-1.5.10:
      Successfully uninstalled kaggle-1.5.10


Streaming output truncated to the last 5000 lines.
  inflating: train/1.2.826.0.1.3680043.8.498.57557504048386802657680113399416280458.jpg  
  inflating: train/1.2.826.0.1.3680043.8.498.57560724131701767501776035124653046817.jpg  
  inflating: train/1.2.826.0.1.3680043.8.498.57562706261298256656000346211938315370.jpg  
  inflating: train/1.2.826.0.1.3680043.8.498.57563228276800171113519710561288251606.jpg  
  inflating: train/1.2.826.0.1.3680043.8.498.57566738057397383506444450455016691068.jpg  
  inflating: train/1.2.826.0.1.3680043.8.498.57571119261069484685650682963089533850.jpg  
  inflating: train/1.2.826.0.1.3680043.8.498.57572767755232342312158107314666054058.jpg  
  inflating: train/1.2.826.0.1.3680043.8.498.57574324320381075854691165962327366197.jpg  
  inflating: train/1.2.826.0.1.3680043.8.498.57575879505596998080282245522650189358.jpg  
  inflating: train/1.2.826.0.1.3680043.8.498.57578355328975593560720792655677959331.jpg  
  inflating: train/1.2.826.0.1.3680043.8.498.5757

In [8]:
import pandas as pd

train = pd.read_csv('train.csv')

train_input = train.drop(['PatientID'], axis=1) #Extract filenames + labels
test_input = test[['StudyInstanceUID']].copy() #Extract filenames

In [11]:
#Reformat variables
train_labels = train.drop(['StudyInstanceUID','PatientID'], axis=1)
train_labels = train_labels.values.tolist()

unvectored = []
for row in train_labels:
  row_temp = []
  for i in range(0, len(row)):
    if row[i]== 1:
      row_temp.append('ETT - Abnormal')
    if row[i]== 1:
      row_temp.append('ETT - Borderline')
    if row[i]== 1:
      row_temp.append('ETT - Normal')
    if row[i]== 1:
      row_temp.append('NGT - Abnormal')
    if row[i]== 1:
      row_temp.append('NGT - Borderline')
    if row[i]== 1:
      row_temp.append('NGT - Incompletely Imaged')
    if row[i]== 1:
      row_temp.append('NGT - Normal')
    if row[i]== 1:
      row_temp.append('CVC - Abnormal')
    if row[i]== 1:
      row_temp.append('CVC - Borderline')
    if row[i]== 1:
      row_temp.append('CVC - Normal')
    if row[i]== 1:
      row_temp.append('Swan Ganz Catheter Present')
  unvectored.append(row_temp)

In [12]:
train_data_input = train[['StudyInstanceUID']].copy()
train_data_input['labels'] = unvectored

def append_ext(fn):
    return fn+".jpg"
train_data_input["StudyInstanceUID"]=train_data_input["StudyInstanceUID"].apply(append_ext)

training_data = train_data_input.iloc[:24000,:] 
validation_data = train_data_input.iloc[24001:,:] 

In [36]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

img_iter_train = train_datagen.flow_from_dataframe(
    training_data,
    directory='train',
    x_col='StudyInstanceUID',
    y_col='labels',
    target_size=(150, 150),
    batch_size=20,
    class_mode='categorical'
)

img_iter_val = val_datagen.flow_from_dataframe(
    validation_data,
    directory='train',
    x_col='StudyInstanceUID',
    y_col='labels',
    target_size=(150, 150),
    batch_size=20,
    class_mode='categorical'
)

Found 24000 validated image filenames belonging to 11 classes.
Found 6082 validated image filenames belonging to 11 classes.


In [25]:
from keras import layers
from keras import models

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu',
input_shape=(150, 150, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(11, activation='sigmoid'))

In [27]:
model.compile(
    loss='binary_crossentropy',
    optimizer='adam'
)

In [28]:
model.fit(img_iter_train,
                   steps_per_epoch=100,
                   epochs=30,
                  validation_data=img_iter_val,
          validation_steps=50)

Epoch 1/30
100/100 [==============================] - 182s 2s/step - loss: 0.0544 - val_loss: 0.0000e+00
Epoch 2/30
100/100 [==============================] - 178s 2s/step - loss: 1.8957 - val_loss: 3.9534e-23
Epoch 3/30
100/100 [==============================] - 186s 2s/step - loss: 9.1248e-24 - val_loss: 3.6091e-21
Epoch 4/30
100/100 [==============================] - 179s 2s/step - loss: 0.0924 - val_loss: 0.0143
Epoch 5/30
100/100 [==============================] - 180s 2s/step - loss: 0.0119 - val_loss: 0.0909
Epoch 6/30
100/100 [==============================] - 176s 2s/step - loss: 0.0579 - val_loss: 0.0019
Epoch 7/30
100/100 [==============================] - 186s 2s/step - loss: 0.0028 - val_loss: 0.0092
Epoch 8/30
100/100 [==============================] - 176s 2s/step - loss: 0.0103 - val_loss: 0.0166
Epoch 9/30
100/100 [==============================] - 176s 2s/step - loss: 0.0218 - val_loss: 7.3649e-04
Epoch 10/30
100/100 [==============================] - 188s 2s/step - l

In [72]:
test = pd.read_csv('sample_submission.csv')

test_files = test[['StudyInstanceUID']].copy()
test_files["StudyInstanceUID"]=test_files["StudyInstanceUID"].apply(append_ext)

test_datagen = ImageDataGenerator(rescale=1./255)
labels = ['null']*3582
test_files['labels'] = labels

img_iter_test = test_datagen.flow_from_dataframe(
    test_files,
    directory='test',
    x_col='StudyInstanceUID',
    y_col = None,
    target_size=(150, 150),
    batch_size=1,
    class_mode=None,
    shuffle = False
)

Found 3582 validated image filenames.


In [76]:
predictions = model.predict(img_iter_test)

In [84]:
submissions = pd.DataFrame(predictions, columns=['ETT - Abnormal',
 'ETT - Borderline',
 'ETT - Normal',
 'NGT - Abnormal',
 'NGT - Borderline',
 'NGT - Incompletely Imaged',
 'NGT - Normal',
 'CVC - Abnormal',
 'CVC - Borderline',
 'CVC - Normal',
 'Swan Ganz Catheter Present'])

In [87]:
submissions.insert(loc = 0, 
          column = 'StudyInstanceUID', 
          value = test['StudyInstanceUID'] ) 

In [89]:
submissions.to_csv('submission.csv')